In [19]:
%%bash

python scClassifier3.py --sup-data-file /home/zengbio/Project/benchmark_broad/bench_broad_dataset.mtx \
                        --sup-label-file /home/zengbio/Project/benchmark_broad/bench_broad_dataset_label.txt \
                        -lr 0.0001 \
                        -n 100 \
                        -bs 1000 \
                        --aux-loss \
                        --validation-fold 10 \
                        --cuda \
                        -ba \
                        --save-model bench_broad_model.pth
                        


Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


1 epoch: avg losses 811683.8728 4552895.9107 0.0000 0.0000 validation accuracy 0.2946 F1 0.0455(macro) 0.1341(weighted) precision 0.0295 recall 0.1000 mcc 0.0000 elapsed 4.9796 seconds
2 epoch: avg losses 623221.1496 3876535.5268 0.0000 0.0000 validation accuracy 0.4142 F1 0.1346(macro) 0.2965(weighted) precision 0.1527 recall 0.1689 mcc 0.2937 elapsed 2.3606 seconds
3 epoch: avg losses 560758.4754 3090010.3571 0.0000 0.0000 validation accuracy 0.6086 F1 0.2452(macro) 0.5024(weighted) precision 0.2408 recall 0.2867 mcc 0.5326 elapsed 2.3646 seconds
4 epoch: avg losses 553891.5603 2135693.9152 0.0000 0.0000 validation accuracy 0.7444 F1 0.3210(macro) 0.6920(weighted) precision 0.3071 recall 0.3449 mcc 0.6769 elapsed 2.3913 seconds
5 epoch: avg losses 547058.4922 1586313.8348 0.0000 0.0000 validation accuracy 0.8024 F1 0.4002(macro) 0.7611(weighted) precision 0.4961 recall 0.4004 mcc 0.7471 elapsed 2.3943 seconds
6 epoch: avg losses 532461.7333 1130925.3929 0.0000 0.0000 validation accur

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
from scipy.io import mmread
from scClassifier3 import scClassifier3
from utils.scdata_cached import SingleCellCached, transform_label2class, label2class_encoder, fn_y_scdata

import torch
from torch.utils.data import DataLoader
import pyro.distributions as dist

from sklearn.metrics import accuracy_score, f1_score, normalized_mutual_info_score, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, Normalizer, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture

import umap
import numpy as np

#import rpy2.robjects.packages as packages
#import rpy2.robjects.lib.ggplot2 as ggplot2
#import rpy2.robjects as ro
#from rpy2.robjects import pandas2ri

#from rpy2.robjects.conversion import localconverter


plt.rcParams["figure.figsize"] = (20, 10)


In [21]:
ModelPath = 'bench_broad_model.pth'


DataPath='/home/zengbio/Project/benchmark_broad/bench_broad_dataset.mtx'
LabelPath='/home/zengbio/Project/benchmark_broad/bench_broad_dataset_label.txt'


In [22]:
# load model
model = torch.load(ModelPath)

In [23]:
# load data
batch_size = 100
use_float64 = False
use_cuda = True

data_cached = SingleCellCached(DataPath, LabelPath, model.class2label, 'valid', use_cuda=use_cuda, use_float64 = use_float64)
data_loader = DataLoader(data_cached, batch_size = batch_size, shuffle = False)



In [24]:
# predict conditions
embeds = []

# use the appropriate data loader
for (xs, ys) in data_loader:
    # use classification function to compute all predictions for each batch
    embed = model.latent_embedding(xs)
    embed = embed.cpu().detach().numpy()
    embeds.append(embed)


embeds = np.concatenate(embeds, 0)


In [25]:
pos = umap.UMAP(metric='cosine').fit_transform(embeds)

In [26]:
pd.DataFrame(pos).to_csv('/home/zengbio/Project/benchmark_broad/bench_broad_dataset_scp_umap.txt')
pd.DataFrame(embeds).to_csv('/home/zengbio/Project/benchmark_broad/bench_broad_dataset_scp_embed.txt')


In [27]:
embeds

array([[-9.0876379e+00, -6.1066532e+00, -3.6808879e+00, ...,
         1.5973413e+01, -4.6784897e+00, -3.0239053e+00],
       [-1.2114469e+01, -3.7997904e+00, -4.6962075e+00, ...,
         1.7967035e+01, -7.5750952e+00, -6.1945009e+00],
       [-9.3488007e+00, -3.6840265e+00, -3.2341595e+00, ...,
         1.4580159e+01, -6.0153532e+00, -4.6962991e+00],
       ...,
       [ 9.9162185e-01, -3.2534668e+00, -4.5300651e-01, ...,
         1.3399389e+00,  8.3013964e-01,  3.3454568e+00],
       [-1.0243944e+00, -5.0764942e+00, -1.6672214e+00, ...,
         5.0269065e+00, -6.5878081e-01,  2.6033833e+00],
       [-1.6373903e-02, -1.8092775e+00, -4.4859463e-01, ...,
         1.3712225e+00, -9.5488280e-02,  1.3453244e+00]], dtype=float32)